In [ ]:
import pandas as pd

In [ ]:
survey_df = pd.read_csv("Surveydata_train.csv")
survey_df_test = pd.read_csv("Surveydata_test .csv")
display(survey_df.head())
display(survey_df_test.head())

In [ ]:
display(survey_df.info())
display(survey_df_test.info())

In [ ]:
display(survey_df.iloc[:,2:17].describe(include = 'all'))
display(survey_df_test.iloc[:,1:16].describe(include = 'all'))

In [ ]:
display(survey_df.isna().sum())
display(survey_df_test.isna().sum())

In [ ]:
travel_df = pd.read_csv("Traveldata_train.csv")
display(travel_df.head())
travel_df_test = pd.read_csv("Traveldata_test.csv")
display(travel_df_test.head())

In [ ]:
display(travel_df.info())
display(travel_df_test.info())

In [ ]:
display(travel_df.iloc[:,1:9].describe(include ='all'))
display(travel_df_test.iloc[:,1:9].describe(include ='all'))

In [ ]:
display(travel_df.isna().sum())
display(travel_df_test.isna().sum())

In [ ]:
# Merge the two datasets

merged_df = pd.merge(survey_df, travel_df, on= 'ID')
display(merged_df.head())

merged_df_test = pd.merge(survey_df_test, travel_df_test, on= 'ID')
display(merged_df_test.head())

In [ ]:
display(merged_df.info())
display(merged_df_test.info())